## Reasoning without Observation

### ReAct 에이전트와의 차이점

#### 1. 토큰 소비 및 실행 시간 감소
- **단일 패스 방식**: 사용할 도구 체인을 한 번에 생성
- **ReACT 방식과의 차이점**: 
  - ReACT는 각 추론 단계마다 LLM 호출 필요
  - 시스템 프롬프트와 이전 단계를 매번 반복 제공해야 함
  - 이로 인한 중복된 프롬프트 발생

#### 2. 미세조정(Fine-tuning) 과정 단순화
- **도구 출력 독립성**: 계획 데이터가 도구의 출력에 의존하지 않음
- **이론적 장점**: 실제 도구를 실행하지 않고도 모델 미세조정 가능

### ReWOO 구성요소
- Planner: Generate the plan in the follwing format:
    - Plan: \<reasoning\>
    - #E1 = Tool[argument for tool]
    - Plan: \<reasoning\>
    - #E2 = Tool[argument for tool with E1 variable substitution]
    - ...
- Worker: executes the tool with the provided arguments.
- Solver: generates the answer for the initial task based on the tool observations.

Planner, Solver는 LLM 호출을 통해 동작. 변수 대체를 통해 Planner LLM 중복 호출 피할 수 있다.

In [1]:
from langchain_teddynote import logging
from dotenv import load_dotenv

load_dotenv()
logging.langsmith("rewoo")

LangSmith 추적을 시작합니다.
[프로젝트명]
rewoo


### Graph State 정의

In [2]:
from typing import List
from typing_extensions import TypedDict

class ReWOO(TypedDict):
    task: str
    plan_string: str
    steps: List
    results: dict
    result: str

### Planner

LLM을 통해 목록 형태로 계획 생성. 각 작업들의 인자는 문자열로 구성, 다른 작업 결과로부터 변수 대체에 사용되는 변수(#E{0-9}+) 포함.<Br>
#E{0-9}+ 변수는 다른 작업 결과로 대체되어 중복 호출 방지.

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

prompt = """
For the following task, make plans that can solve the problem step by step. For each plan, indicate \
which external tool together with tool input to retrieve evidence. You can store the evidence into a \
variable #E that can be called by later tools. (Plan, #E1, Plan, #E2, Plan, ...)

Tools can be one of the following:
(1) Google[input]: Worker that searches results from Google. Useful when you need to find short
and succinct answers about a specific topic. The input should be a search query.
(2) LLM[input]: A pretrained LLM like yourself. Useful when you need to act with general
world knowledge and common sense. Prioritize it when you are confident in solving the problem
yourself. Input can be any instruction.

For example,
Task: Thomas, Toby, and Rebecca worked a total of 157 hours in one week. Thomas worked x
hours. Toby worked 10 hours less than twice what Thomas worked, and Rebecca worked 8 hours
less than Toby. How many hours did Rebecca work?
Plan: Given Thomas worked x hours, translate the problem into algebraic expressions and solve
with Wolfram Alpha. #E1 = WolframAlpha[Solve x + (2x − 10) + ((2x − 10) − 8) = 157]
Plan: Find out the number of hours Thomas worked. #E2 = LLM[What is x, given #E1]
Plan: Calculate the number of hours Rebecca worked. #E3 = Calculator[(2 ∗ #E2 − 10) − 8]

Begin! 
Describe your plans with rich details. Each Plan should be followed by only one #E.

Task: {task}
"""

In [8]:
task = "2024년 노벨 문학상 수상자는 누구야?"
result = model.invoke(prompt.format(task = task))

In [9]:
print(result.content)

Plan: To find out who won the 2024 Nobel Prize in Literature, I will search for the most recent and reliable information on the internet. #E1 = Google[2024 Nobel Prize in Literature winner]

Plan: Retrieve the name of the 2024 Nobel Prize in Literature winner from the search results. #E2 = LLM[Extract the name of the 2024 Nobel Prize in Literature winner from #E1]


In [10]:
import re
from langchain_core.prompts import ChatPromptTemplate

# task 변수(#E) 추출
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
prompt_template = ChatPromptTemplate.from_messages([("user", prompt)])
planner = prompt_template | model

def get_plan(state: ReWOO):
    task = state['task']
    result = planner.invoke({"task": task})
    matches = re.findall(regex_pattern, result.content)
    
    return {"steps": matches, "plan_string": result.content}

In [28]:
test = """
Plan: To find out who won the 2024 Nobel Prize in Literature, I will search for the most recent and reliable information on the internet. #E1 = Google[2024 Nobel Prize in Literature winner]

Plan: Retrieve the name of the 2024 Nobel Prize in Literature winner from the search results. #E2 = LLM[Extract the name of the 2024 Nobel Prize in Literature winner from #E1]"""

In [32]:
regex_pattern = r"Plan:\s*(.+)\s*(#E\d+)\s*=\s*(\w+)\s*\[([^\]]+)\]"
matches = re.findall(regex_pattern, test)
print(matches[0])

('To find out who won the 2024 Nobel Prize in Literature, I will search for the most recent and reliable information on the internet. ', '#E1', 'Google', '2024 Nobel Prize in Literature winner')


### Executor

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()


In [16]:
def _get_current_task(state: ReWOO):
    if "results" not in state or state["results"] is None:
        return 1
    if len(state["results"]) == len(state["steps"]):
        return None
    else:
        return len(state["results"]) + 1
    
def tool_execution(state: ReWOO):
    """Worker node that executes the tools of a given plan."""
    _step = _get_current_task(state)
    _, step_name, tool, tool_input = state["steps"][_step - 1]
    _results = (state["results"] or {}) if "results" in state else {}
    for k, v in _results.items():
        tool_input = tool_input.replace(k,v)
    if tool == "Google":
        result = search.invoke(tool_input)
    elif tool == "LLM":
        result = model.invoke(tool_input)
    else:
        raise ValueError
    _results[step_name] = str(result)
    return {"results": _results}
    

### Solver

In [17]:
solve_prompt = """Solve the following task or problem. To solve the problem, we have made step-by-step Plan and \
retrieved corresponding Evidence to each Plan. Use them with caution since long evidence might \
contain irrelevant information.

{plan}

Now solve the question or task according to provided Evidence above. Respond with the answer
directly with no extra words.

Task: {task}
Response:"""


def solve(state: ReWOO):
    plan = ""
    for _plan, step_name, tool, tool_input in state["steps"]:
        _results = (state["results"] or {}) if "results" in state else {}
        for k, v in _results.items():
            tool_input = tool_input.replace(k, v)
            # 이전 결과 변수로 값 치환
            step_name = step_name.replace(k,v)
        plan += f"{_plan}\n{step_name} = {tool}[{tool_input}]"
    prompt = solve_prompt.format(plan = plan, task=state['task'])
    result = model.invoke(prompt)
    return {"result": result.content}

## Graph 정의

In [18]:
def _route(state):
    _step = _get_current_task(state)
    if _step is None:
        return "solve"
    else:
        return "tool"

In [19]:
from langgraph.graph import END, StateGraph, START

graph = StateGraph(ReWOO)

# 노드 추가
graph.add_node("plan", get_plan)
graph.add_node("tool", tool_execution)
graph.add_node("solve", solve)

# 엣지 추가
graph.add_edge(START, "plan")
graph.add_edge("plan", "tool")
graph.add_edge("solve", END)

# 조건부 엣지 추가
graph.add_conditional_edges(
    "tool",
    _route
)

app = graph.compile()

In [24]:
task = "2024년 노벨 문학상 수상자가 누군지 한국어로 설명해줘."

In [25]:
for s in app.stream({"task": task}):
    print(s)
    print('---')

{'plan': {'plan_string': 'Plan: 먼저, 2024년 노벨 문학상 수상자가 발표되었는지 확인하기 위해 최신 정보를 검색합니다. #E1 = Google[2024 Nobel Literature Prize winner]\n\nPlan: #E1의 검색 결과를 바탕으로 2024년 노벨 문학상 수상자에 대한 정보를 한국어로 요약합니다. #E2 = LLM[Translate and summarize #E1 in Korean]', 'steps': [('먼저, 2024년 노벨 문학상 수상자가 발표되었는지 확인하기 위해 최신 정보를 검색합니다. ', '#E1', 'Google', '2024 Nobel Literature Prize winner'), ('#E1의 검색 결과를 바탕으로 2024년 노벨 문학상 수상자에 대한 정보를 한국어로 요약합니다. ', '#E2', 'LLM', 'Translate and summarize #E1 in Korean')]}}
---
{'tool': {'results': {'#E1': '[{\'url\': \'https://en.wikipedia.org/wiki/2024_Nobel_Prize_in_Literature\', \'content\': \'The 2024 Nobel Prize in Literature was awarded to the South Korean author Han Kang (born 1970) "for her intense poetic prose that confronts historical traumas and exposes the fragility of human life". It was announced by the Swedish Academy in Stockholm, Sweden, on 10 October 2024 and will be awarded on 10 December 2024. [1]She is the first South Korean and first Asian woman to win the 

In [27]:
print(s["solve"]["result"])

2024년 노벨 문학상은 한국 작가 한강에게 수여되었습니다. 그녀는 "역사적 트라우마를 직시하고 인간 삶의 취약성을 드러내는 강렬한 시적 산문"으로 수상자로 선정되었습니다. 한강은 노벨 문학상을 수상한 최초의 한국인이자 아시아 여성입니다.
